In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pandas as pd
import ast
import shutil
from sklearn import model_selection
from tqdm import tqdm
import numpy as np
import pydicom
from tqdm import tqdm
import cv2

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Copying jpeg images in new test dir. 

In [ ]:
!mkdir Test_images
!cp -r ../input/rsna-jpeg-stage-2/Test_images/Test_images/ "Test_images"


In [ ]:
cp -r ../input/flexible-yolov5/flexible-yolov5-main .

Installing requirements

In [ ]:
!pip install -r flexible-yolov5-main/requirements.txt

Copying the yaml files

In [ ]:
!cp "../input/yolov5-pneumonia-yaml/Pneumonia.yaml" "flexible-yolov5-main"
!cp "../input/flexible-yolov5-efficient/Flexible_Yolo_efficientnet.yaml" "flexible-yolov5-main"

In [ ]:
cd ./flexible-yolov5-main

kaggle/working

# Testing

utility functions

In [ ]:
import sys

sys.path.append('.')
from od.models.modules.experimental import *
from od.data.datasets import letterbox
from utils.general import *
from utils.split_detector import SPLITINFERENCE
from utils.torch_utils import *


class Detector(object):
    def __init__(self, pt_path,img_size, conf_thres=0.4, iou_thres=0.3, classes=0, agnostic_nms=False,
                 xcycwh=True, device="cuda"):  #namesfile, #device=0
        self.pt_path = pt_path
        self.img_size = img_size
        self.device = torch.device(device)
        self.model = self.load_model()
        self.conf_thres = conf_thres
        self.iou_thres = iou_thres
        self.classes = classes
        self.agnostic_nms = agnostic_nms
        self.xcycwh = xcycwh
        #self.class_names = self.load_class_names(namesfile)

    def load_model(self):
        model = attempt_load(self.pt_path, map_location=self.device)  # load FP32 model
        return model

    #def load_class_names(self, namesfile):
    #    with open(namesfile, 'r', encoding='utf8') as fp:
     #       class_names = [line.strip() for line in fp.readlines()]
     #   return class_names

    def __call__(self, ori_img, split_width=1, split_height=1):
        if split_width == 1 and split_height == 1:
            bboxes, scores, ids = self.detect_image(ori_img)
        else:
            bboxes = []
            scores = []
            ids = []
            output = self.detect_img_split(image=ori_img, split_width=split_width, split_height=split_height)['data']
            for key in output.keys():
                values = output[key]
                for value in values:
                    x_min = value[0]
                    y_min = value[1]
                    x_max = value[2]
                    y_max = value[3]
                    w = x_max - x_min
                    h = y_max - y_min
                    if self.xcycwh:
                        bboxes.append([x_min + w / 2, y_min + h / 2, w, h])
                    else:
                        bboxes.append(value[:4])
                    scores.append(value[4])
                    ids.append(key)
        return np.asarray(bboxes), np.asarray(scores), np.asarray(ids)

    def detect_image(self, image):
        bboxes = []
        scores = []
        ids = []
        im0s = image
        img = letterbox(im0s, new_shape=self.img_size)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = self.model(img)[0]
        pred = non_max_suppression(pred, self.conf_thres, self.iou_thres, classes=self.classes,
                                   agnostic=self.agnostic_nms)
        for i, det in enumerate(pred):
            if det is not None and len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                for *xyxy, conf, cls in det:
                    x_min = xyxy[0].cpu()
                    y_min = xyxy[1].cpu()
                    x_max = xyxy[2].cpu()
                    y_max = xyxy[3].cpu()
                    score = conf.cpu()
                    clas = cls.cpu()
                    w = x_max - x_min
                    h = y_max - y_min
                    if self.xcycwh:
                        # center coord, w, h
                        bboxes.append([x_min + w / 2, y_min + h / 2, w, h])
                    else:
                        bboxes.append([x_min, y_min, x_max, y_max])
                    scores.append(score)
                    ids.append(clas)
        return np.asarray(bboxes), np.asarray(scores), np.asarray(ids)

    @SPLITINFERENCE(split_width=2, split_height=1)
    def detect_img_split(self, image='', **kwargs):
        outputs_json = {}
        im0s = image
        img = letterbox(im0s, new_shape=self.img_size)[0]
        img = img[:, :, ::-1].transpose(2, 0, 1)
        img = np.ascontiguousarray(img)
        img = torch.from_numpy(img).to(self.device)
        img = img.float()
        img /= 255.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        pred = self.model(img)[0]
        pred = non_max_suppression(pred, self.conf_thres, self.iou_thres, classes=self.classes,
                                   agnostic=self.agnostic_nms)
        for i, det in enumerate(pred):
            if det is not None and len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0s.shape).round()
                for *xyxy, conf, cls in det:
                    x_min = xyxy[0].cpu()
                    y_min = xyxy[1].cpu()
                    x_max = xyxy[2].cpu()
                    y_max = xyxy[3].cpu()
                    score = conf.cpu()
                    clas = cls.cpu()
                    if clas in outputs_json:
                        outputs_json[clas].append([x_min, y_min, x_max, y_max, score])
                    else:
                        outputs_json[clas] = [[x_min, y_min, x_max, y_max, score]]
        return {'data': outputs_json}


Inference of an image folder

In [ ]:

'''def multiple_detection(pt_path = "", img_root = "", save_dir = "", size = 512):
    
    model = Detector(pt_path, size, xcycwh=False)
    imgs = os.listdir(imgs_root)
    #save_dir =  '/kaggle/working/output2/'
    label_dir = os.path.join(save_dir,'labels')
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    print(save_dir)
    print(label_dir)
    for img in imgs:
        im = cv2.imread(os.path.join(imgs_root, img))
        bboxes, scores, ids = model.detect_image(im)
        for idx in range(bboxes.shape[0]):
            bbox = bboxes[idx].astype(int)
            score = scores[idx]
            cv2.rectangle(im, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2, 1)


            text = "Pneumonia: {:.4f}".format(score)
            cv2.putText(im, text, (bbox[0], bbox[1] - 5), cv2.FONT_HERSHEY_SIMPLEX,
                              0.5, (0, 0, 255), 2)
            cv2.imwrite(os.path.join(save_dir, img), im)

            if not os.path.exists(label_dir):
                      os.makedirs(label_dir)
            with open((os.path.join(label_dir, img[:-4] + '.txt')), 'a+') as f:
                      f.write("0" + ' ' + str(bbox[0]) + ' ' + str(bbox[1]) + ' ' + str(bbox[2]) + ' ' + str(bbox[3]) +' '+ str(score) + '\n')


'''

# Multiple image detection (images are saved in an output folder)

In [ ]:
'''pt_path = '../../input/yolov5-efficientnetb7/best.pt'

imgs_root = "/kaggle/working/Test_images/Test_images/"
save_dir =  '/kaggle/working/output_test3/'
multiple_detection(pt_path, imgs_root,  save_dir, 512)'''

In [ ]:
def display_single_img_detect(pt_path="", size = 512, img_path="" ):
    
   # pt_path = '../../input/yolov5-efficientnetb7/best.pt'
    model = Detector(pt_path, size, xcycwh=False, device='cpu')
    
    im = cv2.imread(img_path)
    bboxes, scores, ids = model.detect_image(im)
    for idx in range(bboxes.shape[0]):
        bbox = bboxes[idx].astype(int)
        score = scores[idx]
        cv2.rectangle(im, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 255), 2, 1)


        text = "Pneumonia: {:.4f}".format(score)
        cv2.putText(im, text, (bbox[0], bbox[1] - 5), cv2.FONT_HERSHEY_SIMPLEX,
                          0.5, (0, 0, 255), 2)
        #cv2.imwrite(os.path.join(save_dir, img), im)
        print(score)
    # displaying image
    
    plt.imshow(im)
   # cv2.waitKey(0)
    plt.show()
      

# Displaying single image detection (images are not saved in folder)

In [ ]:
imgs_root = "/kaggle/working/Test_images/Test_images"
img_path = "00ad18b7-06ee-4c4d-abca-14bdf814e8b2.jpg"
img_full_path = os.path.join(imgs_root, img_path)
print(img_full_path)
pt_path = '../../input/yolov5-efficientnetb7/best.pt'



display_single_img_detect(pt_path, 512, img_full_path)